#### CLASSWISE MODEL TUNING CODE
##### Author : Kunal Kalwankar

In [ ]:
#Imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import torch
import json
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import os
!pip install simpleT5
from simplet5 import SimpleT5
import time

In [ ]:
#importing pre deicded testids for all the topics. These were split 90-10 only once for all iterations and saved
os.listdir('/kaggle/input/cnndm-tagged-subset')
data_init = pd.read_csv('/kaggle/input/cnndm-tagged-subset/masterdata.csv')
t1 = pd.read_csv('/kaggle/input/testids/testids_1_to_5.csv')
t2 = pd.read_csv('/kaggle/input/testids/testids_2_to_5.csv')
t3 = pd.read_csv('/kaggle/input/testids/testids_3_to_5.csv')
t4 = pd.read_csv('/kaggle/input/testids/testids_4_to_5.csv')
t5 = pd.read_csv('/kaggle/input/testids/testids_5_to_5.csv')
t = pd.concat([t1,t2,t3,t4,t5])

#maximum epochs
MAX_EPOCHS = 3

#import knn tags or topic tags based on autoNLP
knn = pd.read_csv('/kaggle/input/knn5tags/KNN5.csv')[['classname']]

#check distribution
print(knn['classname'].value_counts())

#join the tags to dataset
data_init = pd.concat([data_init,knn],axis=1)

In [ ]:
#Iterative training that goes on for hours. One model is trained and saved per value in the list.
#Each checkpoint after

ls = [0,1,2,3,4]
for i in ls:
    data = data_init[data_init['classname']==i]
    
    #Hyperpaarmeters
    MAX_LEN = 512
    SUMMARY_LEN = 150
    
    #select train and test
    test = data[data['id'].isin(list(t['TESTIDS']))]
    train = data[~data['id'].isin(list(t['TESTIDS']))]
    TRAINING_SIZE = train.shape[0]
    training_df = train.copy()
    training_article_ls = list(training_df['article'])
    training_highlight_ls = list(training_df['highlights'])
    df = pd.DataFrame(columns=['target_text','source_text'])
    
    # manipulate inputs
    df['target_text'] = training_highlight_ls
    df['source_text'] = ['summarize: '+item for item in training_article_ls]
    model = SimpleT5()
    model.from_pretrained(model_type="t5", model_name="t5-small")
    t1 = time.time()
    
    #70-30 splits
    model.train(train_df=df[0:(int)(0.7*TRAINING_SIZE)],
                eval_df=df[(int)(0.7*TRAINING_SIZE):TRAINING_SIZE], 
                source_max_token_len=MAX_LEN, 
                target_max_token_len=SUMMARY_LEN, 
                batch_size=8, max_epochs=MAX_EPOCHS, use_gpu=True)
    print(time.time()-t1)
    
    # save model 
    l = os.listdir('/kaggle/working/outputs')  
    modelname = [g for g in l if 'simplet5-epoch-2' in g]
    modelname = [i for i in modelname if i not in used_models]
    used_models.append(modelname[0])
    modelname=modelname[0]
    model.load_model("t5","outputs/"+modelname, use_gpu=True)
    print(modelname)
    articles = list(test['article'])
    
    #prediction function
    def prediction(x):
        return model.predict(x)
    store = [prediction(g) for g in articles]
    test['predictions'] = store
    
    #Save file with predictions to pass to metric calculator
    test.to_csv('t5small_knn5_5epochs_'+str(i)+'_.csv')